In [1]:
!pip install autotrain-advanced

In [2]:
!autotrain setup

2023-08-07 11:02:39.746715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl


In [4]:
!autotrain dreambooth --help

2023-08-07 11:05:42.185068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: autotrain <command> [<args>] dreambooth
       [-h]
       --model
       MODEL
       [--revision REVISION]
       [--tokenizer TOKENIZER]
       --image-path
       IMAGE_PATH
       [--class-image-path CLASS_IMAGE_PATH]
       --prompt
       PROMPT
       [--class-prompt CLASS_PROMPT]
       [--num-class-images NUM_CLASS_IMAGES]
       [--class-labels-conditioning CLASS_LABELS_CONDITIONING]
       [--prior-preservation]
       [--prior-loss-weight PRIOR_LOSS_WEIGHT]
       --output
       OUTPUT
       [--seed SEED]
       --resolution
       RESOLUTION
       [--center-crop]
       [--train-text-encoder]
       [--batch-size BATCH_SIZE]
       [--sample-batch-size SAMPLE_BATCH_SIZE]
       [--epochs EPOCHS]
       [--num-steps NUM_STEPS]
       [--checkpointing-steps CHECKPOINTING_STEPS]
       [--resume-from-checkpoint RESUME_FROM_CHECKPOINT]
       [

In [5]:
!autotrain dreambooth --model stabilityai/stable-diffusion-xl-base-1.0 --output output --image-path gman --prompt "photo of sks guy" --resolution 1024 --batch-size 1 --num-steps 500 --fp16 --gradient-accumulation 4 --lr 1e-4

2023-08-07 11:20:36.898709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='gman', class_image_path=None, prompt='photo of sks guy', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, output='output', seed=42, resolution=1024, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=500, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=None, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=None, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=None

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights("model/", weight_name="pytorch_lora_weights.safetensors")

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
)
refiner.to("cuda")


prompt = "a portrait of sks guy, pixar, cartoon, 3d, headshots, fantasy, 4k, uhd"

for seed in range(10):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image.images[0]
    image.save(f"images/{seed}.png")
    image = refiner(prompt=prompt, generator=generator, image=image)
    image = image.images[0]
    image.save(f"images_refined/{seed}.png")